# Welcome to the heart of the project, the code. Through this Jupyter notebook, I'll share the thought process behind most of the code.

>We begin by sending a request through the Kaggle API and pulling the database we're interested in from Kaggle:

In [1]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [2]:
api.dataset_download_file(
    dataset='alperenmyung/social-media-advertisement-performance',
    file_name='ad_campaign_db.sqlite',
    path='.'
)

Dataset URL: https://www.kaggle.com/datasets/alperenmyung/social-media-advertisement-performance


True

>Then we load the SQL magic extension and establish a connection to the downloaded database.

In [3]:
%load_ext sql

In [4]:
import sqlite3
conn = sqlite3.connect('ad_campaign_db.sqlite')

In [5]:
%sql sqlite:///ad_campaign_db.sqlite

>The following was used to fix an error caused by the default styling setting in magic SQL commands.

In [6]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

>Lets see the tables:

In [7]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///ad_campaign_db.sqlite
Done.


name
users
campaigns
ads
ad_events


>A quick peek into one of the tables:

In [8]:
%sql SELECT * FROM users LIMIT 5;

 * sqlite:///ad_campaign_db.sqlite
Done.


user_id,user_gender,user_age,age_group,country,location,interests
a2474,Female,24,18-24,United Kingdom,New Mariomouth,"fitness, health"
141e5,Male,21,18-24,Germany,Danielsfort,"food, fitness, lifestyle"
34db0,Male,27,25-34,Australia,Vincentchester,"fashion, news"
20d08,Female,28,25-34,India,Lisaport,"health, news, finance"
9e830,Male,28,25-34,United States,Brownmouth,"health, photography, lifestyle"


In [9]:
import pandas as pd

>Now let’s see all the columns of every table in the database to plan our analysis:

In [10]:
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table'", conn).name

for t in tables:
    cols = pd.read_sql_query(f"SELECT name FROM pragma_table_info('{t}')", conn)
    print(f"Table: {t}")
    print(cols['name'].to_list(), "\n")

Table: users
['user_id', 'user_gender', 'user_age', 'age_group', 'country', 'location', 'interests'] 

Table: campaigns
['campaign_id', 'name', 'start_date', 'end_date', 'duration_days', 'total_budget'] 

Table: ads
['ad_id', 'campaign_id', 'ad_platform', 'ad_type', 'target_gender', 'target_age_group', 'target_interests'] 

Table: ad_events
['event_id', 'ad_id', 'user_id', 'timestamp', 'day_of_week', 'time_of_day', 'event_type'] 



>Now, we will answer questions generated by ChatGPT:

---
"How many users are there in total, and how are they distributed by gender?"

In [42]:
%%sql
SELECT
    COUNT(DISTINCT CASE WHEN user_gender = "Male" THEN user_id END) AS men,
    COUNT(DISTINCT CASE WHEN user_gender = "Female" THEN user_id END) As women
FROM users;

 * sqlite:///ad_campaign_db.sqlite
Done.


men,women
5521,3439


---
"How many unique users engaged in each type of ad event (`event_type`)"

In [44]:
%%sql
SELECT 
    event_type, COUNT(DISTINCT user_id) AS unique_users
FROM ad_events
GROUP BY event_type;

 * sqlite:///ad_campaign_db.sqlite
Done.


event_type,unique_users
Click,9772
Comment,3339
Impression,9950
Like,6959
Purchase,1835
Share,1789


---
Which day of the week had the highest number of ad clicks (`event_type = 'click'`) across all campaigns? Show the day and the number of clicks.

In [55]:
%%sql
SELECT 
    ae.day_of_week,
    COUNT(*) AS total_clicks
FROM ad_events ae
JOIN ads a 
    ON ae.ad_id = a.ad_id
JOIN campaigns c 
    ON a.campaign_id = c.campaign_id
WHERE ae.event_type = 'Click'
GROUP BY ae.day_of_week
ORDER BY total_clicks DESC
LIMIT 1;

 * sqlite:///ad_campaign_db.sqlite
Done.


day_of_week,total_clicks
Friday,5828


---
Which ad platform had the most total ad events?

In [66]:
%%sql
SELECT
    a.ad_platform, COUNT(ae.ad_id) AS total_events
FROM ads a, ad_events ae
WHERE a.ad_id = ae.ad_id
GROUP BY a.ad_platform
ORDER BY total_events DESC;

 * sqlite:///ad_campaign_db.sqlite
Done.


ad_platform,total_events
Facebook,254096
Instagram,145904


---
Which age group has the highest number of users targeted by ads?

In [70]:
%%sql
SELECT
    a.target_age_group, COUNT(DISTINCT ae.user_id) AS unique_users
FROM ads a
JOIN ad_events ae ON a.ad_id = ae.ad_id
JOIN users u ON ae.user_id = u.user_id AND a.target_age_group = u.age_group
GROUP BY a.target_age_group
ORDER BY unique_users DESC;

 * sqlite:///ad_campaign_db.sqlite
Done.


target_age_group,unique_users
25-34,4127
18-24,3115
35-44,1454


---
How many ads are there per ad type?

In [79]:
%%sql
SELECT ad_type, COUNT(ad_type) AS frequency FROM ads
GROUP BY ad_type
ORDER BY frequency DESC;

 * sqlite:///ad_campaign_db.sqlite
Done.


ad_type,frequency
Stories,64
Image,52
Carousel,51
Video,33


---
For each ad platform, what is the average duration (in days) of campaigns that ran ads on that platform?

In [81]:
%%sql
SELECT a.ad_platform, AVG(c.duration_days)
FROM ads a
JOIN campaigns c ON c.campaign_id = a.campaign_id
GROUP BY a.ad_platform;

 * sqlite:///ad_campaign_db.sqlite
Done.


ad_platform,AVG(c.duration_days)
Facebook,66.48031496062993
Instagram,66.15068493150685


---
Which country has the highest number of users who clicked on ads?

In [84]:
%%sql
SELECT u.country, COUNT(ae.event_type) AS clicks
FROM users u
JOIN ad_events ae ON u.user_id = ae.user_id
WHERE event_type='Click'
GROUP BY u.country
ORDER BY clicks DESC LIMIT 1;

 * sqlite:///ad_campaign_db.sqlite
Done.


country,clicks
United States,12292


---
For each campaign, what’s the percentage of ad events that were clicks compared to all events?

In [95]:
%%sql
SELECT 
  c.name, 
  100.0 * SUM(CASE WHEN ae.event_type = 'Click' THEN 1 ELSE 0 END) / COUNT(*) AS click_percentage
FROM campaigns c
JOIN ads a ON c.campaign_id = a.campaign_id
JOIN ad_events ae ON a.ad_id = ae.ad_id
GROUP BY c.name
ORDER BY click_percentage DESC
LIMIT 5;

 * sqlite:///ad_campaign_db.sqlite
Done.


name,click_percentage
Campaign_12_Q3,10.913140311804009
Campaign_22_Q3,10.711462450592885
Campaign_39_Q3,10.477127397934087
Campaign_19_Winter,10.454091816367265
Campaign_30_Winter,10.440864243786686


---
Identify the top 3 campaigns with the highest number of unique users in the age group “18-24” who interacted with their ads.

In [102]:
%%sql
SELECT
    c.name, COUNT(DISTINCT u.user_id) AS unique_users
FROM campaigns c
JOIN ads a ON c.campaign_id = a.campaign_id
JOIN ad_events ae ON a.ad_id = ae.ad_id
JOIN users u ON ae.user_id = u.user_id
WHERE u.age_group = "18-24"
GROUP BY c.name
ORDER BY unique_users DESC
LIMIT 3;

 * sqlite:///ad_campaign_db.sqlite
Done.


name,unique_users
Campaign_42_Summer,2516
Campaign_38_Q3,2510
Campaign_20_Winter,2491


---
Now, we'll turn these database tables into dataframes and analyze them using BI tools:

In [103]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///ad_campaign_db.sqlite")

In [104]:
users = pd.read_sql_table('users', con=engine)
campaigns = pd.read_sql_table('campaigns', con=engine)
ads = pd.read_sql_table('ads', con=engine)
ad_events = pd.read_sql_table('ad_events', con=engine)

In [105]:
users.head(3)

,user_id,user_gender,user_age,age_group,country,location,interests
0,a2474,Female,24,18-24,United Kingdom,New Mariomouth,"fitness, health"
1,141e5,Male,21,18-24,Germany,Danielsfort,"food, fitness, lifestyle"
2,34db0,Male,27,25-34,Australia,Vincentchester,"fashion, news"


In [106]:
for name, df in {'users': users, 'campaigns': campaigns, 'ads': ads, 'ad_events': ad_events}.items():
    print(f"{name} duplicates: {sum(df.duplicated())}, null values: {sum(df.isnull().sum())}")

users duplicates: 0, null values: 0
campaigns duplicates: 0, null values: 0
ads duplicates: 0, null values: 0
ad_events duplicates: 0, null values: 0


As mentioned in the README file, these data tables do not contain any duplicated rows or missing values.

Next up, we shall export these dataframes as CSV files.

In [107]:
for name, df in {'users': users, 'campaigns': campaigns, 'ads': ads, 'ad_events': ad_events}.items():
    df.to_csv(f"{name}.csv", index=False)